In [ ]:
conda create --name py2 python=2.7


In [ ]:
conda activate py2


In [ ]:
curl -O https://raw.githubusercontent.com/ialbert/booleannet/master/examples/projects/LGL/LGL-simulation.py


In [ ]:
curl -O https://raw.githubusercontent.com/ialbert/booleannet/master/examples/projects/LGL/LGL.txt

In [ ]:
python LGL-simulation.py

In [ ]:
curl -O https://raw.githubusercontent.com/ialbert/booleannet/master/examples/projects/LGL/LGL-plot.py
    

In [ ]:
curl -O https://raw.githubusercontent.com/ialbert/booleannet/master/examples/projects/LGL/LGL-stimulation.py

In [ ]:
python LGL-stimulation.py

In [ ]:
python LGL-plot.py

In [ ]:
"""
Bordetella Bronchiseptica  simulation
Takes about 30 seconds to run
"""
import boolean2
from boolean2 import Model, util 
from boolean2.plde import helper


# the overrides.py module contains all node overrides
import overrides

# setting up simulation parameters
FULLT = 50
STEPS = FULLT*50

#
# the helper functions allow for easy parameter extraction from a csv files
#

# parameters for concentration, decay and treshold for various nodes
CONC_PARAMS = helper.read_parameters( 'Bb-concentration.csv' )

# parameters for compartment ratios and fluctuations
COMP_PARAMS = helper.read_parameters( 'Bb-compartmental.csv' )

# use data from the sixth row (it is zero based counting!) in the file
CONC = CONC_PARAMS[5]
COMP = COMP_PARAMS[5]

# helper function that Binds the local override to active COMP parameter
def local_override( node, indexer, tokens ):
    return overrides.override( node, indexer, tokens, COMP )

#
# there will be two models, one for WT and the other for a BC knockout
#
wt_text = file('Bb.txt').read()
bc_text = boolean2.modify_states( text=wt_text, turnoff= [ "BC"  ] )

model1 = Model( text=wt_text, mode='plde' )
model2 = Model( text=bc_text, mode='plde' )

model1.OVERRIDE = local_override
model2.OVERRIDE = local_override

model1.initialize( missing = helper.initializer( CONC )  )
model2.initialize( missing = helper.initializer( CONC )  )

# see localdefs for all function definitions
model1.iterate( fullt=FULLT, steps=STEPS, localdefs='localdefs' )
model2.iterate( fullt=FULLT, steps=STEPS, localdefs='localdefs' )

# saves the simulation resutls into a file
data = [ model1.data, model2.data, model1.t ]

# it is a binary save ( pickle )
util.bsave(data, 'Bb-run.bin' )

In [ ]:
"""
Bordetella Bronchiseptica  simulation
- plotting the results
"""

from pylab import *
from boolean2 import util

def skip( data, step=10):
    "Takes every Nth point from a list or dictionary values"
    if type(data) == type([]):
        return data[::step]
    
    out = dict()
    for key, values in list(data.items()):
        values = values[::step]
        out[key] = values
    return out

def make_plot():
    run1, run2, t = util.bload( 'Bb-run.bin' )
    
    # take every 10th point
    step = 20
    run1 = skip(run1, step=step)
    run2 = skip(run2, step=step)
    t = skip(t, step=step)

    nodes = "EC PIC C PH IL12I IL12II".split()
    
    subplot(121)

    # drawing these first so that the symbols are under the other ones
    p1 = plot(t, run2['EC'], 'r^-', ms=7 )
    p2 = plot(t, run2['PIC'], 'r^-', ms=7 )
    
    p7 = plot(t, run1['EC'], 'b.-', ms=5 )
    p8 = plot(t, run1['PIC'], 'b--', ms=5 )
    
    xlabel( 'Time' )
    ylabel( 'Concentration' )
    title ( 'Innate Immune Response' )
    legend( [p1, p2, p7, p8], 'DEL-EC DEL-PIC WT-EC WT-PIC'.split(), loc='best')
    
    subplot(122)
   
    p3 = plot(t, run2['C'], 'r^-', ms=7 )
    p4 = plot(t, run2['PH'], 'ro-', ms=7 )
    p5 = plot(t, run2['IL12I'], 'r.-', ms=7 )
    p6 = plot(t, run2['IL12II'], 'rs-', ms=7 )
    
    p9 = plot(t, run1['C'], 'bo-', ms=5 )
    p10 = plot(t, run1['PH'] , 'bD-', ms=5 )
    p11 = plot(t, run1['IL12I'], 'b.-', ms=5 )
    p12 = plot(t, run1['IL12II'], 'b^-', ms=5 )
    
    xlabel( 'Time' )
    ylabel( 'Concentration' )
    title ( 'Adaptive Immune Response' )
    legend( [p3, p4, p5, p6, p9, p10, p11, p12], 'DEL-C DEL-PH DEL-IL12I DEL-IL12II WT-C WT-PH WT-IL12I WT-IL12II'.split(), loc='best')
    
if __name__ == '__main__':
    figure(num = None, figsize=(14, 7), dpi=80, facecolor='w', edgecolor='k')
    make_plot()
    savefig('Figure3.png')
    show()